In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from IPython import get_ipython

# Define the URLs for the links
url1 = "/lab/"
url2 = "/voila/render/ui/conn.ipynb"
url3 = "/voila/render/ui/qprof_main.ipynb"

# Define the button labels
button_label1 = "Go to Home Page"
button_label2 = "Go to Connection"
button_label3 = "Go Back"

# Define HTML links for buttons
button_html1 = f"<a href='{url1}' style='text-decoration: none; padding: 8px; background-color: #007bff; color: white; border-radius: 5px; margin-right: 10px;'><i class='fa fa-home'></i></a>"
button_html2 = f"<a href='{url2}' style='text-decoration: none; padding: 8px; background-color: #007bff; color: white; border-radius: 5px;'>{button_label2}</a>"
button_html3 = f"<a href='{url3}' style='text-decoration: none; padding: 8px; background-color: #007bff; color: white; border-radius: 5px; margin-right: 5px;'><i class='fa fa-arrow-left'></i></a>"


# Create HTML widgets
html_button1 = widgets.HTML(value=button_html1)
html_button2 = widgets.HTML(value=button_html2)
html_button3 = widgets.HTML(value=button_html3)


# Display buttons
display(widgets.HBox([html_button3, html_button1, html_button2]))


________

In [ ]:
import os
import logging

log_folder = 'logs'
if not os.path.exists(log_folder):
    os.makedirs(log_folder)
logging.basicConfig(filename='logs/jupyter_log.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
logging.info('[Query Profile Tree Page] Importing Libraries')
import pickle
import ipywidgets as widgets
from datetime import datetime, timezone
import time
import random
import string
import numpy as np
import verticapy as vp
import plotly.graph_objects as go
from collections import defaultdict, Counter
import textwrap
from verticapy.performance.vertica import QueryProfilerInterface, QueryProfiler, QueryProfilerComparison
logging.info('[Query Profile Tree Page] Successfully Imported Libraries ')
output_initial = widgets.Output()
global qprof
global qprof2
comparison = False
with output_initial:
    %store -r query_val
    %store -r key_val
    %store -r target_schema_val
    %store -r transaction_val
    %store -r statement_val
    %store -r file_val
    %store -r target_schema_val_2
    %store -r key_val_2
    %store -r transaction_statement_list_val
    %store -r control_query_val
    %store -r transaction_comp1_val
    %store -r statement_comp1_val
    %store -r target_schema_comp1_val
    %store -r key_comp1_val
    %store -r query_comp1_val
    %store -r session_control_1_val
    %store -r transaction_comp2_val
    %store -r statement_comp2_val
    %store -r target_schema_comp2_val
    %store -r key_comp2_val
    %store -r query_comp2_val
    %store -r session_control_2_val

    if "file_val" in globals():
        if not "key_val" in globals():
            characters = string.ascii_letters + string.digits  # includes 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
            key_val = ''.join(random.choice(characters) for _ in range(15))
        logging.info(f'[Query Profile Tree Page] Importing profile using target_schema as {target_schema_val}, key_id as {key_val}, filename as{file_val}')
        qprof = QueryProfiler.import_profile(target_schema=target_schema_val,
            key_id = key_val,
            filename=file_val,
            auto_initialize = False                                          
        )
        logging.info(f'[Query Profile Tree Page] Creating qprof object using target_schema as {target_schema_val}, key_id as {key_val if "key_val" in globals() else None}.')
        qprof = QueryProfilerInterface(
            target_schema = target_schema_val,
            key_id = key_val,
            check_tables = False,
        )
    elif "target_schema_val" in globals():
        logging.info(f'[Query Profile Tree Page] Creating qprof object using target_schema as {target_schema_val}, key_id as {key_val}. [Query Profile Tree Page]')
        qprof = QueryProfilerInterface(
            target_schema = target_schema_val,
            key_id = key_val,
            check_tables = False,
        )
    elif "query_val" in globals():
        logging.info(f'[Query Profile Tree Page] Creating qprof object using query as "{query_val}" target_schema as {target_schema_val_2 if "target_schema_val_2" in globals() else None}, key_id as {key_val_2 if "key_val_2" in globals() else None}')
        qprof = QueryProfilerInterface(
            query_val,
            target_schema = target_schema_val_2 if "target_schema_val_2" in globals() else None,
            key_id = key_val_2 if "key_val_2" in globals() else None,
            check_tables = False,
            session_control = control_query_val if "control_query_val" in globals() else None,
        )
    elif "transaction_val" in globals():
        logging.info(f'[Query Profile Tree Page] Creating qprof object using transactions as {eval(transaction_val), eval(statement_val)}, target_schema as {target_schema_val_2 if "target_schema_val_2" in globals() else None}, key_id as{key_val_2 if "key_val_2" in globals() else None}')
        qprof = QueryProfilerInterface(
            transactions = (eval(transaction_val), eval(statement_val)),
            target_schema = target_schema_val_2 if "target_schema_val_2" in globals() else None,
            key_id = key_val_2 if "key_val_2" in globals() else None,
            check_tables = False,
        )
    elif "transaction_statement_list_val" in globals():
        logging.info(f'[Query Profile Tree Page] Creating qprof object using transactions as {eval(transaction_statement_list_val)}, target_schema as {target_schema_val_2 if "target_schema_val_2" in globals() else None}, key_id as{key_val_2 if "key_val_2" in globals() else None}')
        qprof = QueryProfilerInterface(
            transactions = eval(transaction_statement_list_val),
            target_schema = target_schema_val_2 if "target_schema_val_2" in globals() else None,
            key_id = key_val_2 if "key_val_2" in globals() else None,
            check_tables = False,
        )

    else:
        comparison = True
        if "transaction_comp1_val" in globals():
            qprof1 = QueryProfilerInterface(
                transactions = (eval(transaction_comp1_val),eval(statement_comp1_val)),
                check_tables = False,
            )
        elif "target_schema_comp1_val" in globals():
            qprof1 = QueryProfilerInterface(
                target_schema = target_schema_comp1_val,
                key_id = key_comp1_val,
                check_tables = False,
            )
        elif "query_comp1_val" in globals():
            qprof1 = QueryProfilerInterface(
                query_comp1_val,
                check_tables = False,
                session_control = session_control_1_val if "session_control_1_val" in globals() else None,
            )
        if "transaction_comp2_val" in globals():  # Changed from transaction_comp1_val to transaction_comp2_val
            qprof2 = QueryProfilerInterface(  # Changed from qprof1 to qprof2
                transactions = (eval(transaction_comp2_val),eval(statement_comp2_val)),  # Changed from transaction_comp1_val to transaction_comp2_val
                check_tables = False,
            )
        elif "target_schema_comp2_val" in globals():  # Changed from target_schema_comp1_val to target_schema_comp2_val
            qprof2 = QueryProfilerInterface(  # Changed from qprof1 to qprof2
                target_schema = target_schema_comp2_val,  # Changed from target_schema_comp1_val to target_schema_comp2_val
                key_id = key_comp2_val,  # Changed from key_comp1_val to key_comp2_val
                check_tables = False,
            )
        elif "query_comp2_val" in globals():  # Changed from query_comp1_val to query_comp2_val
            qprof2 = QueryProfilerInterface(  # Changed from qprof1 to qprof2
                query_comp2_val,  # Changed from query_comp1_val to query_comp2_val
                check_tables = False,
                session_control = session_control_2_val if "session_control_2_val" in globals() else None,  # Changed from session_control_1_val to session_control_2_val
            )
        qprof_comparison = QueryProfilerComparison(qprof1, qprof2)
         

In [ ]:
# Loading display settings
#Default
vp.set_option("max_rows", 20000)
vp.set_option("insert_comma_numbers", True)
# Saved
with output_initial:
    # Attempt to load stored variables
    try:
        %store -r count_widget_val
    except KeyError:
        pass
    try:
        %store -r footer_widget_val
    except KeyError:
        pass
    try:
        %store -r comma_widget_val
    except KeyError:
        pass
    try:
        %store -r percent_bar_widget_val
    except KeyError:
        pass
    try:
        %store -r max_cell_width_widget_val
    except KeyError:
        pass
    try:
        %store -r max_columns_widget_val
    except KeyError:
        pass
    try:
        %store -r max_rows_widget_val
    except KeyError:
        pass

    # Set options using the loaded or default values
    try:
        if count_widget_val is not None:
            vp.set_option("count_on", count_widget_val)
        if footer_widget_val is not None:
            vp.set_option("footer_on", footer_widget_val)
        if comma_widget_val is not None:
            vp.set_option("insert_comma_numbers", comma_widget_val)
        else:
            vp.set_option("insert_comma_numbers", True)
        if percent_bar_widget_val is not None:
            vp.set_option("percent_bar", percent_bar_widget_val)
        if max_cell_width_widget_val is not None:
            vp.set_option("max_cellwidth", max_cell_width_widget_val)
        if max_columns_widget_val is not None:
            vp.set_option("max_columns", max_columns_widget_val)
        if max_rows_widget_val is not None:
            vp.set_option("max_rows", max_rows_widget_val)
        else:
            vp.set_option("max_rows", 20000)
    except Exception as e:
        print(e)


In [ ]:
button_qprof_save = widgets.Button(description="Save Qprof Information & Report", layout=widgets.Layout(width='250px'))
output_export = widgets.Output()

def click_export_query(_):
    output_export.clear_output(wait=True)
    with output_export:
        print("Saving Query. It may take a while...")
        try:
            logging.info('[Query Profile Tree Page] Trying to export qprof object to a "qprof.tar" file')
            qprof.export_profile(filename="qprof.tar")
            print("Ready to Download")
            display(download_output)
            display(btn)
        except Exception as e:
            logging.info(f'[Query Profile Tree Page] Failed to export qprof because of this error:{e}')
            print("\033[91mUnsuccessful.\033[0m")
            print("Error:", e)
        print("\n Press the button below to generate and download the HTML report...")
        display(btn_report)


button_qprof_save.on_click(click_export_query)

qprof_save = widgets.VBox([button_qprof_save, output_export])

In [ ]:
import tarfile
from ipywidgets import Output, Button
from base64 import b64encode

download_output = Output()


def trigger_download(data, filename, file_type='application/octet-stream'):
    logging.info(f'[Query Profile Tree Page] Trying to download file')
    content_b64 = b64encode(data).decode()
    data_url = f'data:{file_type};base64,{content_b64}'
    js_code = f"""
        var a = document.createElement('a');
        a.setAttribute('download', '{filename}');
        a.setAttribute('href', '{data_url}');
        a.click();
    """
    with download_output:
        clear_output()
        display(HTML(f'<script>{js_code}</script>'))

btn = Button(description='Click to download the tar file', layout=widgets.Layout(width='250px'))

def download_qprof(e=None):
    # Assuming 'qprof.tar' is stored in the file path 'qprof_path'
    qprof_path = 'qprof.tar' 
    with open(qprof_path, 'rb') as file:
        qprof_data = file.read()
    trigger_download(qprof_data, 'qprof.tar')

btn.on_click(download_qprof)

In [ ]:
def trigger_download_report(data, filename, file_type='text/html'):
    logging.info(f'[Query Profile Tree Page] Trying to download HTML report.')
    content_b64 = b64encode(data).decode()
    data_url = f'data:{file_type};base64,{content_b64}'
    js_code = f"""
        var a = document.createElement('a');
        a.setAttribute('download', '{filename}');
        a.setAttribute('href', '{data_url}');
        a.click();
    """
    with output_export:
        clear_output()
        display(HTML(f'<script>{js_code}</script>'))

download_report_output = widgets.Output()
btn_report = Button(description='Click to download HTML report', layout=widgets.Layout(width='250px'))

def download_html(e=None):
    global qprof
    with output_export:
        print("Preparing files....")
        # Define the HTML content
        html_content = qprof.to_html()
        html_data = html_content.encode('utf-8')
        trigger_download_report(html_data, 'sample.html')      
        print("Done")


btn_report.on_click(download_html)

In [ ]:
top_accordion = widgets.Accordion(children=[qprof_save], titles=('Downloads',))
if comparison:
    button_qprof_save.disabled = True
display(top_accordion)

In [ ]:
logging.info(f'[Query Profile Tree Page] Completed the script to automatically display the tree')

def create_tooltip(text):
    # Create an info icon using HTML
    info_icon = widgets.HTML(
        value=f'<span style="font-size: 25px; color: #007bff; cursor: pointer;" title="{text}">&#9432;</span>'
    )
    return info_icon

In [ ]:
button_other_info = widgets.Button(description="Get Detailed Info", layout=widgets.Layout(padding='0 0 10px 0'))

def on_button_clicked_other_info(b):
    button_other_info.disabled = True
    table_1.clear_output(wait=True)
    table_2.clear_output(wait=True)
    try:
        transaction_statement_list = qprof.transactions
        logging.info(f'[Query Profile Tree Page] The list of transaction and statement ids are: {transaction_statement_list}')
        # Construct the WHERE clause dynamically
        where_clause = " OR ".join([f"(transaction_id = {elem[0]} AND statement_id = {elem[1]})" for elem in transaction_statement_list])
        
        # Construct the query
        query = f"""
        SELECT transaction_id, statement_id, node_name, event_description, time, duration_us/1000000 AS duration_sec
        FROM dc_slow_events
        WHERE {where_clause}
        ORDER BY duration_sec DESC
        LIMIT 500;
        """
        query_2 = f"""
        SELECT *
        FROM dc_optimizer_events
        WHERE {where_clause};
        """
        logging.info(f'[Query Profile Tree Page] Trying to get the DC SLOW EVENTS table')
        res_1 = vp.vDataFrame(f"""{query}""")
        res_2 = vp.vDataFrame(f"""{query_2}""")
        with table_1:
            res_1.idisplay()
        with table_2:
            res_2.idisplay()
    except Exception as e:
        with table_1:
            display(widgets.HTML(f"<p style='color:red'>The following error occured: </p> <p> {e}</p>"))
        with table_2:
            display(widgets.HTML(f"<p style='color:red'>The following error occured: </p> <p> {e}</p>"))
    button_other_info.disabled = False
        
table_1 = widgets.Output()
table_2 = widgets.Output()
tables = widgets.Tab()
tables.children = [table_1, table_2]
tables.titles = ["Slow Events", "Optimizer Events"]
other_info = widgets.VBox([button_other_info, tables])

button_other_info.on_click(on_button_clicked_other_info)

button_get_queries = widgets.Button(description="Get Detailed Info", layout=widgets.Layout(padding='0 0 10px 0'))
output_get_queries = widgets.Output()

def on_button_clicked_get_queries(b):
    button_get_queries.disabled = True
    output_get_queries.clear_output(wait=True)
    with output_get_queries:
        try:
            vdf = qprof.get_queries()[qprof.get_queries().get_columns(exclude_columns = "is_current")]
            vdf["qduration"].rename("duration (seconds)")
            vdf.idisplay()
        except Exception as e:
            widgets.VBox([widgets.HTML(f"<p style='color:red'>The following error occured: </p> <p> {e}</p>")])
    button_get_queries.disabled = False

button_get_queries.on_click(on_button_clicked_get_queries)

In [ ]:
# Query Engine Table tab

engine_tab_summary_1 = widgets.Output()



selected_checkboxes = ["exec_time_us", "prod_rows", "proc_rows", "clock_time_us"]

# Function to update based on selections
def on_checkbox_change(change):
    global selected_checkboxes, summary_vdf, granularity_level
    
    # If the checkbox is being checked
    if change['new']:
        # If we already have 6 checkboxes selected, prevent this one from being checked
        if len(selected_checkboxes) >= 6:
            change['owner'].value = False
            return
        selected_checkboxes.append(change['owner'].description)
    else:
        # If the checkbox is being unchecked, remove it from the list
        selected_checkboxes.remove(change['owner'].description)

    # Update the dataframe based on selected checkboxes
    detailed_update_table(node_name=summary_node_name_dropdown.value, 
                          path_id=summary_path_id_dropdown.value, 
                          localplan_id=summary_localplan_id_dropdown.value, 
                          operator_name=summary_operator_name_dropdown.value)

# Selecting the granularity of the summary table
engine_table_granularity_switch = widgets.RadioButtons(
    options=['Across paths', 'Across nodes'],
    value='Across paths',  # Defaults to 'Pivot Table of metrics'
    # description='Select Granularity Level:',
    disabled=False
)
granularity_level = 2
engine_table_granularity_switch_title = widgets.HBox([widgets.HTML('<b> Select granularity level:</b>'), 
                                                      create_tooltip("""
The granularity level of the summary table. 
Across paths: The summary table for each path. 
Across nodes: The summary table for each node. All rows are summed and times are averaged
"""
        )
    ]
)
def on_granularity_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        if change['new'] == 'Across nodes':
            granularity_level = 1
            summary_node_name_dropdown.disabled = True
            summary_localplan_id_dropdown.disabled = True
            summary_operator_name_dropdown.disabled = False
            summary_baseplan_id_dropdown.disabled = False
            display_vdf_across_nodes(path_id=summary_path_id_dropdown.value, baseplan_id=summary_baseplan_id_dropdown.value, operator_name=summary_operator_name_dropdown.value)
        elif change['new'] == 'Across paths':
            granularity_level = 2
            summary_node_name_dropdown.disabled = True
            summary_localplan_id_dropdown.disabled = True
            summary_operator_name_dropdown.disabled = True
            display_vdf_across_paths(path_id=summary_path_id_dropdown.value)
    



engine_table_granularity_switch.observe(on_granularity_change)

try:
    # summary_vdf = qprof._get_vdf_summary()[0]["node_name", "path_id", "localplan_id", "operator_name", "exec_time_us", "est_rows", "rle_prod_rows"]
    vdf_across_paths = None
    vdf_across_nodes = None

    def display_vdf_across_paths(path_id="All"):
        if vdf_across_paths is None:
            return
        vdf = vdf_across_paths.copy()
        with engine_tab_summary_1:
            engine_tab_summary_1.clear_output(wait=True)
            if path_id != "All":
                vdf = vdf[vdf["path_id"] == path_id]
            # display(widgets.HTML(vdf._repr_html_()))
            display(vdf.idisplay())

    def display_vdf_across_nodes(path_id="All", baseplan_id="All", operator_name="All"):
        if vdf_across_nodes is None:
            return
        vdf = vdf_across_nodes.copy()
        with engine_tab_summary_1:
            engine_tab_summary_1.clear_output(wait=True)
            if path_id != "All":
                vdf = vdf[vdf["path_id"] == path_id]
            if baseplan_id != "All":
                vdf = vdf[vdf["baseplan_id"] == baseplan_id]
            if operator_name != "All":
                vdf = vdf[vdf["operator_name"] == operator_name]
            # display(widgets.HTML(vdf._repr_html_()))
            display(vdf.idisplay())
    # summary_update_table(node_name = "All", path_id = "All", localplan_id = "All", opeartor_name = "All")
    


    summary_node_name_dropdown = widgets.Dropdown(
        options=['All'],
        description='Select Node Name:',
        style={'description_width': 'initial'},
        value='All',
        disabled=True
    )
    summary_path_id_dropdown = widgets.Dropdown(
        options=['All'],
        description='Select Path ID:',
        style={'description_width': 'initial'},
        value='All'
    )
    summary_localplan_id_dropdown = widgets.Dropdown(
        options=['All'],
        description='Select Local Plan ID:',
        style={'description_width': 'initial'},
        value='All',
        disabled=True
    )
    summary_operator_name_dropdown = widgets.Dropdown(
        options=['All'],
        description='Select Operator Name:',
        style={'description_width': 'initial'},
        value='All'
    )
    summary_baseplan_id_dropdown = widgets.Dropdown(
        options=['All'],
        description='Select Base Plan ID:',
        style={'description_width': 'initial'},
        value='All',
    )
    engine_table_summary_controls = widgets.VBox([summary_node_name_dropdown, summary_path_id_dropdown, summary_localplan_id_dropdown, summary_baseplan_id_dropdown,summary_operator_name_dropdown])
    
    def summary_on_node_name_dropdown_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            selected_key = change['new']
            if selected_key == 'All':
                summary_path_id_dropdown.options = ['All'] + sorted(summary_vdf_main["path_id"].distinct())
            else:
                vdf = summary_vdf_main[summary_vdf_main["node_name"]==selected_key]
                summary_path_id_dropdown.options = ['All'] + sorted(vdf["path_id"].distinct())
            summary_path_id_dropdown.value = 'All'
            if engine_table_granularity_switch.value == 'Across nodes':
                display_vdf_across_nodes()
            else:
                display_vdf_across_paths()
            
            detailed_update_table(node_name = selected_key)
    
    summary_node_name_dropdown.observe(summary_on_node_name_dropdown_change, names='value')
    
    def summary_on_path_id_dropdown_change(change):
        if not summary_node_name_dropdown.value == "All":
            vdf=summary_vdf_main[summary_vdf_main["node_name"]==summary_node_name_dropdown.value]
            summary_localplan_id_dropdown.options = ['All'] + sorted(vdf["localplan_id"].distinct())
            summary_operator_name_dropdown.options = ['All'] + sorted(vdf["operator_name"].distinct())
        else:
            vdf = summary_vdf_main
        if change['type'] == 'change' and change['name'] == 'value':
            selected_key = change['new']
            if selected_key == 'All':
                summary_localplan_id_dropdown.options = ['All'] + sorted(vdf["localplan_id"].distinct())
                summary_operator_name_dropdown.options = ['All'] + sorted(vdf["operator_name"].distinct())
            else:                
                vdf = vdf[vdf["path_id"]==selected_key]
                summary_localplan_id_dropdown.options = ['All'] + sorted(vdf["localplan_id"].distinct())
                summary_operator_name_dropdown.options = ['All'] + sorted(vdf["operator_name"].distinct())
            summary_localplan_id_dropdown.value = 'All'
            if engine_table_granularity_switch.value == 'Across nodes':
                display_vdf_across_nodes(path_id = selected_key)
            else:
                display_vdf_across_paths(path_id = selected_key)
            detailed_update_table(node_name = summary_node_name_dropdown.value, path_id = selected_key)
            
    summary_path_id_dropdown.observe(summary_on_path_id_dropdown_change, names='value')
    
    def summary_on_localplan_id_dropdown_change(change):
        if not summary_node_name_dropdown.value == "All":
            vdf=summary_vdf_main[summary_vdf_main["node_name"]==summary_node_name_dropdown.value]
            summary_operator_name_dropdown.options = ['All'] + sorted(vdf["operator_name"].distinct())
        else:
            vdf = summary_vdf_main
        if not summary_path_id_dropdown.value == "All":
            vdf=vdf[vdf["path_id"]==summary_path_id_dropdown.value]
        if change['type'] == 'change' and change['name'] == 'value':
            selected_key = change['new']
            if selected_key == 'All':
                summary_operator_name_dropdown.options = ['All'] + sorted(vdf["operator_name"].distinct())
            else:
                vdf = vdf[vdf["localplan_id"]==selected_key]
                summary_operator_name_dropdown.options = ['All'] + sorted(vdf["operator_name"].distinct())
            summary_operator_name_dropdown.value = 'All'
            # if engine_table_granularity_switch.value == 'Across nodes':
            #     display_vdf_across_nodes(path_id = summary_path_id_dropdown.value, localplan_id = selected_key)
            # else:
            #     display_vdf_across_paths(path_id = summary_path_id_dropdown.value)
            detailed_update_table(node_name = summary_node_name_dropdown.value,path_id = summary_path_id_dropdown.value,localplan_id = selected_key)

    summary_localplan_id_dropdown.observe(summary_on_localplan_id_dropdown_change, names='value')   
    
    def summary_on_baseplan_id_dropdown_change(change):
        if not summary_node_name_dropdown.value == "All":
            vdf=vdf_across_nodes[vdf_across_nodes["node_name"]==summary_node_name_dropdown.value]
            summary_operator_name_dropdown.options = ['All'] + sorted(vdf["operator_name"].distinct())
        else:
            vdf = vdf_across_nodes
        if change['type'] == 'change' and change['name'] == 'value':
            selected_key = change['new']
            if selected_key == 'All':
                summary_operator_name_dropdown.options = ['All'] + sorted(vdf["operator_name"].distinct())
            else:
                vdf = vdf[vdf["baseplan_id"]==selected_key]
                summary_operator_name_dropdown.options = ['All'] + sorted(vdf["operator_name"].distinct())
            summary_operator_name_dropdown.value = 'All'
            if engine_table_granularity_switch.value == 'Across nodes':
                display_vdf_across_nodes(path_id = summary_path_id_dropdown.value, baseplan_id = selected_key)
            else:
                display_vdf_across_paths(path_id = summary_path_id_dropdown.value)
            detailed_update_table(node_name = summary_node_name_dropdown.value, path_id = summary_path_id_dropdown.value, baseplan_id = selected_key)
    
    summary_baseplan_id_dropdown.observe(summary_on_baseplan_id_dropdown_change, names='value')   
    
    def summary_on_operator_name_dropdown_change(change):
        if not summary_node_name_dropdown.value == "All":
            vdf=summary_vdf_main[summary_vdf_main["node_name"]==summary_node_name_dropdown.value]
        else:
            vdf = summary_vdf_main
        if not summary_path_id_dropdown.value == "All":
            vdf=vdf[vdf["path_id"]==summary_path_id_dropdown.value]
        if not summary_localplan_id_dropdown.value == "All":
            vdf=vdf[vdf["localplan_id"]==summary_localplan_id_dropdown.value]
        if change['type'] == 'change' and change['name'] == 'value':
            selected_key = change['new']
            if engine_table_granularity_switch.value == 'Across nodes':
                display_vdf_across_nodes(path_id=summary_path_id_dropdown.value, baseplan_id=summary_baseplan_id_dropdown.value, operator_name=selected_key)
            else:
                display_vdf_across_paths(path_id=summary_path_id_dropdown.value)
            detailed_update_table(node_name=summary_node_name_dropdown.value, path_id=summary_path_id_dropdown.value, localplan_id=summary_localplan_id_dropdown.value, operator_name=selected_key)
    summary_operator_name_dropdown.observe(summary_on_operator_name_dropdown_change, names='value')   

    engine_table_granularity_title_switch = widgets.VBox([engine_table_granularity_switch_title, engine_table_granularity_switch], layout=widgets.Layout(padding='0 0 0 10px'))
    engine_tab_summary_all = widgets.VBox([
        widgets.HBox([engine_table_summary_controls, widgets.HTML("<div></div>"), engine_table_granularity_title_switch]),
        engine_tab_summary_1
    ])

except Exception as e:
    engine_tab_summary_all = widgets.HTML(f"<p style='color:red'>The following error occured: </p> <p> {e}</p>")
try:
    drop_down_query_select = qprof.query_select_dropdown
except Exception as e:
    drop_down_query_select = widgets.HTML(f"<p style='color:red'>The following error occured: </p> <p> {e}</p>")


# Tab for detailed Executio Engine Table
engine_table_detailed_selection_title = widgets.HBox([widgets.HTML('<b> Select detailed view:</b>'), 
                                                      create_tooltip("""
Pivot Table: The detailed view of the query execution engine with selected metrics.
Raw Table: The raw table of the EXECUTION_ENGINE_PROFILES.
"""
        )
    ]
)
engine_table_detailed_selection_switch = widgets.RadioButtons(
    options=['Pivot Table', 'Raw Table'],
    value='Pivot Table',
    disabled=False
)
engine_table_detailed_selection_title_switch = widgets.VBox([
    engine_table_detailed_selection_title,
    engine_table_detailed_selection_switch
], layout=widgets.Layout(padding='0 0 0 10px'))

def on_detailed_view_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        detailed_update_table(
            node_name=summary_node_name_dropdown.value,
            path_id=summary_path_id_dropdown.value,
            localplan_id=summary_localplan_id_dropdown.value,
            operator_name=summary_operator_name_dropdown.value
        )
engine_table_detailed_selection_switch.observe(on_detailed_view_change, names='value')
engine_table_detailed_widget = widgets.Output(layout=widgets.Layout(width='1400px'))
try:
    query = (f""" 
    select node_name, path_id, localplan_id, operator_id, operator_name, counter_name, counter_value from v_monitor.execution_engine_profiles
    where transaction_id = {qprof.transaction_id} and statement_id = {qprof.statement_id} order by 1,2,3,4;
    """)
    query = qprof._replace_schema_in_query(query)
    detailed_vdf = vp.vDataFrame(query)
    
    def detailed_update_table(node_name="All", path_id="All", localplan_id="All", operator_name="All"):
        global detailed_vdf, summary_vdf_main, selected_checkboxes
        
        with engine_table_detailed_widget:
            engine_table_detailed_widget.clear_output(wait=True)
            if engine_table_detailed_selection_switch.value == "Raw Table":
                vdf = detailed_vdf.copy()
            elif engine_table_detailed_selection_switch.value == "Pivot Table":
                vdf = summary_vdf_main.copy()
                cols = ["node_name", "path_id", "localplan_id", "operator_name"]
                cols.extend(selected_checkboxes)
                vdf = vdf[cols]
            if path_id != "All":
                vdf = vdf[vdf["path_id"] == path_id]
            if node_name != "All":
                vdf = vdf[vdf["node_name"] == node_name]
            if localplan_id != "All":
                vdf = vdf[vdf["localplan_id"] == localplan_id]
            if operator_name != "All":
                vdf = vdf[vdf["operator_name"] == operator_name]
            # display(widgets.HTML(vdf._repr_html_()))
            vdf.idisplay()
    
    
except Exception as e:
    engine_table_detailed_widget = widgets.HTML(f"<p style='color:red'>The following error occured: </p> <p> {e}</p>") 

engine_table_detailed_top = widgets.HBox([engine_table_summary_controls, engine_table_detailed_selection_title_switch])
global engine_table_detailed
engine_table_detailed = widgets.VBox([
    engine_table_detailed_top,
    engine_table_detailed_widget,
    # checkboxes, engine_table_detailed_widget
])


engine_table_tab = widgets.Tab()
engine_table_tab.children = [engine_tab_summary_all, engine_table_detailed]
engine_table_tab.titles = ["Summary", "Detailed"]


def on_tab_change(change):
    if change['new'] == 0:  # Summary tab
        update_summary_dropdowns()
    else:  # Detailed tab
        enable_all_dropdowns()

def update_summary_dropdowns():
    if engine_table_granularity_switch.value == 'Across paths':
        summary_node_name_dropdown.disabled = True
        summary_path_id_dropdown.disabled = False
        summary_localplan_id_dropdown.disabled = True
        summary_operator_name_dropdown.disabled = True
        summary_baseplan_id_dropdown.disabled = True
    else:  # 'Across nodes'
        summary_node_name_dropdown.disabled = True
        summary_path_id_dropdown.disabled = False
        summary_localplan_id_dropdown.disabled = False
        summary_operator_name_dropdown.disabled = False
        summary_baseplan_id_dropdown.disabled = False
def enable_all_dropdowns():
    summary_node_name_dropdown.disabled = False
    summary_path_id_dropdown.disabled = False
    summary_localplan_id_dropdown.disabled = False
    summary_operator_name_dropdown.disabled = False
    summary_baseplan_id_dropdown.disabled = True # It is not used in the Detailed tab.

engine_table_tab.observe(on_tab_change, names='selected_index')
# Button to calcualte Query Execution Engine data
engine_table_button = widgets.Button(description = "Fetch data", layout=widgets.Layout(margin='10px 0 10px 0'))
def on_engine_table_button_click(b):
    engine_table_button.disabled = True
    global summary_vdf, summary_vdf_main, vdf_across_paths, vdf_across_nodes, checkboxes
    
    try:
        # Adjust the column list based on the current granularity level
        if granularity_level == 2:
            columns = ["path_id", "thread_count", "exec_time_us", "prod_rows", "proc_rows", "clock_time_us", "mem_all_b"]
        elif granularity_level == 1:
            columns = ["path_id", "baseplan_id", "operator_name", "thread_count", "exec_time_us", "prod_rows", "proc_rows", "clock_time_us", "mem_all_b"]
        else:
            columns = ["node_name", "path_id", "localplan_id", "operator_name", "thread_count", "exec_time_us", "prod_rows", "proc_rows", "clock_time_us", "mem_all_b"]

        vdf_across_paths = qprof._get_vdf_summary()[2]["path_id", "thread_count", "exec_time_us", "prod_rows", "proc_rows", "clock_time_us", "mem_all_b"]
        vdf_across_nodes = qprof._get_vdf_summary()[1]["path_id", "baseplan_id", "operator_name", "thread_count", "exec_time_us", "prod_rows", "proc_rows", "clock_time_us", "mem_all_b"]
        summary_vdf_main = qprof._get_vdf_summary()[0]
        summary_vdf = summary_vdf_main["node_name", "path_id", "localplan_id", "operator_name", "thread_count", "exec_time_us", "prod_rows", "proc_rows", "clock_time_us", "mem_all_b"]

        # Update dropdowns
        summary_node_name_dropdown.options = ['All'] + sorted(summary_vdf["node_name"].distinct())
        summary_path_id_dropdown.options = ['All'] + sorted(summary_vdf["path_id"].distinct())
        summary_localplan_id_dropdown.options = ['All'] + sorted(summary_vdf["localplan_id"].distinct())
        summary_operator_name_dropdown.options = ['All'] + sorted(summary_vdf["operator_name"].distinct())
        summary_baseplan_id_dropdown.options = ['All'] + sorted(vdf_across_nodes["baseplan_id"].distinct())

        # Initial table updates
        display_vdf_across_paths()
        detailed_update_table()

        tags = sorted(qprof.get_qexecution_report(return_useful_cols=True))

        # Create checkboxes
        checkboxes = [
            widgets.Checkbox(value=(tag in ["exec_time_us", "prod_rows", "proc_rows", "clock_time_us", "mem_all_b"]), description=tag) 
            for tag in tags
        ]

        # Attach the observer to each checkbox
        for cb in checkboxes:
            cb.observe(on_checkbox_change, names='value')

        engine_table_detailed.children = [
            engine_table_detailed_top,
            widgets.HBox([engine_table_detailed_widget, widgets.VBox(checkboxes)]),
        ]

    except Exception as e:
        with engine_tab_summary_1:
            engine_tab_summary_1.clear_output(wait=True)
            display(widgets.HTML(f"<p style='color:red'>The following error occurred: </p> <p> {e}</p>"))
    engine_table_button.disabled = False

engine_table_button.on_click(on_engine_table_button_click)


engine_table_all = widgets.VBox([drop_down_query_select,engine_table_button,engine_table_tab])

In [ ]:
# Plots

plot_tab_out = widgets.Output()
plot_tab_button = widgets.Button(description="Generate plots", layout=widgets.Layout(padding='0 0 10px 0'))

def on_button_clicked_plot_tab_button(b):
    plot_tab_button.disabled = True
    plot_tab_out.clear_output(wait=True)
    with plot_tab_out:
        try:
            query_plan_profile = qprof.get_qplan_profile()
            query_plan_profile_plot = go.FigureWidget(data=[query_plan_profile.data[i] for i in range(len(query_plan_profile.data))],)
            
            query_execution = qprof.get_qexecution()
            query_execution_plot = go.FigureWidget(data=[query_execution.data[i] for i in range(len(query_execution.data))],)
            
            plot_tabs = widgets.Tab()
            plot_tabs.children = [query_plan_profile_plot, query_execution_plot]
            plot_tabs.titles = ["Query Plan Profile", "Query Execution Plot"]
            display(plot_tabs)
        except Exception as e:
            plot_tabs = widgets.VBox([widgets.HTML(f"<p style='color:red'>The following error occured: </p> <p> {e}</p>")])
            display(plot_tabs)
    plot_tab_button.disabled = False

plot_tab_button.on_click(on_button_clicked_plot_tab_button)
plot_tabs_main = widgets.VBox([plot_tab_button, plot_tab_out])

In [ ]:
# Tab for Query Statistics

button_query_stats = widgets.Button(description="Run statistics", layout=widgets.Layout(margin='10px 0 0 0'))
output_query_stats = widgets.Output()

# Display charts using widgets
output_optimize_stat = widgets.Output()
output_network_stat = widgets.Output()
output_execution_stat = widgets.Output()

vertical_line = widgets.HTML(value='<div style="border-left: 1px solid #F2F1EF; height: 100%;"></div>')
vertical_line_gap = widgets.HTML(value='<div style="border-left: 1px solid white; height: 100%;"></div>')

apply_button = widgets.Button(
    description='APPLY',
    button_style='success',  # 'success' (green), 'info' (light blue), 'warning' (yellow), 'danger' (red)
    tooltip='Apply filters',
    icon='check'  # (FontAwesome names without the `fa-` prefix)
)

def update_table_widget_professional(data, node_filter, category_filter, type_filter):
    # Filter data based on selected node, category, and type
    filtered_data = []
    for priority_list in data:
        filtered_priority_list = [item for item in priority_list 
                                  if (node_filter == 'All' or item[0] == node_filter) and 
                                     (category_filter == 'All' or item[1] == category_filter) and
                                     (type_filter == 'All' or item[2] == type_filter)]
        filtered_data.append(filtered_priority_list)
    return create_table_widget_professional(tuple(filtered_data))

all_tests = None
node_dropdown = None
category_dropdown = None
type_dropdown = None

def on_button_clicked_query_stats(b):
    global all_tests, node_dropdown, category_dropdown, type_dropdown
    button_query_stats.disabled = True
    output_query_stats.clear_output(wait=True)
    try:
        all_tests = qprof.main_tests()
        node_dropdown = widgets.Dropdown(description="Node", options=["All"] + sorted({val[0] for test in all_tests for val in test}), value='All')
        category_dropdown = widgets.Dropdown(description="Category", options=["All"] + sorted({val[1] for test in all_tests for val in test}), value='All', layout=widgets.Layout(margin='10px 0 0 0'))
        type_dropdown = widgets.Dropdown(description="Type", options=["All"] + sorted({val[2] for test in all_tests for val in test}), value='All', layout=widgets.Layout(margin='10px 0 0 0'))

        with output_query_stats:           
            optimize_chart = create_pie_chart("OPTIMIZER", analyze_stats_data(all_tests)[0])
            network_chart = create_pie_chart("NETWORK", analyze_stats_data(all_tests)[1])
            execution_chart = create_pie_chart("EXECUTION", analyze_stats_data(all_tests)[2])
            
            with output_optimize_stat:
                display(optimize_chart)
            
            with output_network_stat:
                display(network_chart)
            
            with output_execution_stat:
                display(execution_chart)
            display(widgets.HTML(value="<hr>"))
            display(widgets.HTML(value="<h1> Alerts </h1>"))
            
            # Arrange charts horizontally
            display(widgets.HBox([output_optimize_stat, vertical_line_gap, vertical_line, output_network_stat, vertical_line_gap, vertical_line, output_execution_stat]))

            display(widgets.HTML(value="<hr>"))
            display(widgets.HTML(value="<h1> Query events </h1>"))
            filter_title = widgets.HTML(value='<h2>Filters</h2>')
            
            filters_box = widgets.VBox([filter_title, node_dropdown, category_dropdown, type_dropdown, widgets.HTML("<br>"), apply_button], 
                                       layout=widgets.Layout(
                                           border='1px solid #ddd', 
                                           padding='10px', 
                                           border_radius='5px', 
                                           box_shadow='0 2px 5px rgba(0,0,0,0.1)'
                                       ))
            display(filters_box)
            with out_apply_button:
                display(update_table_widget_professional(all_tests, node_dropdown.value, category_dropdown.value, type_dropdown.value))

    except Exception as e:
        widgets.VBox([widgets.HTML(f"<p style='color:red'>The following error occurred: </p> <p> {e}</p>")])
    button_query_stats.disabled = False

out_apply_button = widgets.Output()

def on_button_clicked_apply_button(b):
    out_apply_button.clear_output(wait=True)
    with out_apply_button:
        display(update_table_widget_professional(all_tests, node_dropdown.value, category_dropdown.value, type_dropdown.value))

apply_button.on_click(on_button_clicked_apply_button)
button_query_stats.on_click(on_button_clicked_query_stats)


# Constants for column visibility
SHOW_NODE = True
SHOW_CATEGORY = True
SHOW_TYPE = True
SHOW_DESCRIPTION = True
SHOW_REMEDY = True
SHOW_ACTION = True

# Column widths (adjust as needed)
COLUMN_WIDTHS = {
    'NODE': '15%',
    'CATEGORY': '10%',
    'TYPE': '20%',
    'DESCRIPTION': '20%',
    'REMEDY': '20%',
    'ACTION': '15%'
}

def wrap_text(text, max_width=45):
    return '<br>'.join(textwrap.wrap(text, max_width))

def create_table_widget_professional(input_tuple):
    style = {'description_width': 'initial'}
    layout = widgets.Layout(width='100%')
    
    # Custom CSS for the entire table and header
    custom_css = """
    <style>
    .professional-table {
        font-family: Arial, sans-serif;
        border-collapse: collapse;
        width: 100%;
        border: 1px solid #ddd;
        border-radius: 8px;
        overflow: hidden;
    }
    .professional-header {
        background: linear-gradient(to bottom, #f8f8f8 0%, #e8e8e8 100%);
        color: #333;
        font-weight: bold;
        text-align: left;
        padding: 12px 15px;
        border-bottom: 2px solid #ddd;
    }
    .professional-header th {
        padding: 12px 15px;
    }
    .professional-row {
        transition: background-color 0.3s;
    }
    .professional-row:hover {
        filter: brightness(95%);
    }
    .professional-cell {
        padding: 12px 15px;
        border-bottom: 1px solid #ddd;
        word-wrap: break-word;
        overflow-wrap: break-word;
        white-space: normal;
    }
    .professional-button {
        border: none;
        color: white;
        text-align: center;
        text-decoration: none;
        display: inline-block;
        font-size: 14px;
        margin: 4px 2px;
        transition-duration: 0.4s;
        cursor: pointer;
        border-radius: 4px;
    }
    .professional_button_enabled:hover {
        background-color: #45a049;
    }
    .professional_button_enabled {
        background-color: #32cee6;
    }
    .professional_button_disabled {
        background-color: #d4d8d9;
    }
    .professional_button_disabled:hover {
        background-color: #d4d8d9;
    }
    </style>
    """
    display(HTML(custom_css))
    
    # Dynamically create header based on visible columns
    header_columns = []
    visible_columns = []
    if SHOW_NODE: 
        header_columns.append(f"<th style='width: {COLUMN_WIDTHS['NODE']}'>Node</th>")
        visible_columns.append('NODE')
    if SHOW_CATEGORY: 
        header_columns.append(f"<th style='width: {COLUMN_WIDTHS['CATEGORY']}'>Category</th>")
        visible_columns.append('CATEGORY')
    if SHOW_TYPE: 
        header_columns.append(f"<th style='width: {COLUMN_WIDTHS['TYPE']}'>Type</th>")
        visible_columns.append('TYPE')
    if SHOW_DESCRIPTION: 
        header_columns.append(f"<th style='width: {COLUMN_WIDTHS['DESCRIPTION']}'>Description</th>")
        visible_columns.append('DESCRIPTION')
    if SHOW_REMEDY: 
        header_columns.append(f"<th style='width: {COLUMN_WIDTHS['REMEDY']}'>Remedy</th>")
        visible_columns.append('REMEDY')
    if SHOW_ACTION: 
        header_columns.append(f"<th style='width: {COLUMN_WIDTHS['ACTION']}'>Action</th>")
        visible_columns.append('ACTION')
    
    header = widgets.HTML(
        value=f"""
        <table class="professional-table">
            <thead>
                <tr class="professional-header">
                    {"".join(header_columns)}
                </tr>
            </thead>
        </table>
        """
    )

    row_widgets = []
    input_tuple = input_tuple[::-1]
    for priority, rows in enumerate(input_tuple):
        if priority == 0:
            bg_color = '#ffcccb'  # light red/pink for high priority
        elif priority == 1:
            bg_color = '#ffd7b5'  # light orange for medium priority
        else:
            bg_color = '#c7f2c2' #'#f0f0f0'  # neutral gray for low priority

        for i, row in enumerate(rows):
            node, category, type_, description, remedy, action = row + [''] * (6 - len(row))  # Pad with empty strings if needed
            
            row_cells = []
            for column in visible_columns:
                if column == 'NODE':
                    row_cells.append(widgets.HTML(value=f'<div style="background-color: {bg_color}; width: 100%; height: 100%;"><div class="professional-cell">{wrap_text(node)}</div></div>', layout=widgets.Layout(width=COLUMN_WIDTHS['NODE'])))
                elif column == 'CATEGORY':
                    row_cells.append(widgets.HTML(value=f'<div style="background-color: {bg_color}; width: 100%; height: 100%;"><div class="professional-cell">{wrap_text(category)}</div></div>', layout=widgets.Layout(width=COLUMN_WIDTHS['CATEGORY'])))
                elif column == 'TYPE':
                    row_cells.append(widgets.HTML(value=f'<div style="background-color: {bg_color}; width: 100%; height: 100%;"><div class="professional-cell">{wrap_text(type_)}</div></div>', layout=widgets.Layout(width=COLUMN_WIDTHS['TYPE'])))
                elif column == 'DESCRIPTION':
                    row_cells.append(widgets.HTML(value=f'<div style="background-color: {bg_color}; width: 100%; height: 100%;"><div class="professional-cell">{wrap_text(description)}</div></div>', layout=widgets.Layout(width=COLUMN_WIDTHS['DESCRIPTION'])))
                elif column == 'REMEDY':
                    row_cells.append(widgets.HTML(value=f'<div style="background-color: {bg_color}; width: 100%; height: 100%;"><div class="professional-cell">{wrap_text(remedy)}</div></div>', layout=widgets.Layout(width=COLUMN_WIDTHS['REMEDY'])))
                elif column == 'ACTION':
                    remedy_button = widgets.Button(
                        description="Take Action" if action else "No Action",
                        disabled = True if action=="" else False,
                        layout=widgets.Layout(width='100%', height = '40px'),
                        style=style
                    )
                    remedy_button.add_class('professional-button')
                    if action:
                        remedy_button.add_class('professional_button_enabled')
                    else:
                        remedy_button.add_class('professional_button_disabled')
                    remedy_button.on_click(lambda b, action=action: remedy_action(action)(b))
                    row_cells.append(widgets.HBox([remedy_button], layout=widgets.Layout(width=COLUMN_WIDTHS['ACTION'], justify_content='center', align_items='center', background_color=bg_color)))
            
            row = widgets.HBox(row_cells, layout=widgets.Layout(margin='0', padding='0', width='100%'))
            row_widgets.append(row)

    table = widgets.VBox([header] + row_widgets, 
                         layout=widgets.Layout(margin='20px 0',
                                               box_shadow='0 0 10px rgba(0,0,0,0.1)'))
    
    return table



def remedy_action(sql_action):
    def action_handler(b):
        ip = get_ipython()
        
        if sql_action:
            ip.user_ns['sql_action'] = sql_action  # Add the variable to the user namespace
            ip.run_line_magic('store', 'sql_action')  # Store the variable
        else:
            ip.run_line_magic('store', '-d sql_action')  # Delete the stored variable
        with out_apply_button:
            notebook_url = '/voila/render/ui/sql_action.ipynb'
            display(HTML(f"<script>window.open('{notebook_url}');</script>"))
    
    return action_handler


def parse_tuple(data_tuple):
    # Define the priorities for each list
    priorities = [1, 2, 3]

    # Initialize the result list
    result = []

    # Loop through each list in the tuple and its corresponding priority
    for priority, data_list in zip(priorities, data_tuple):
        for item in data_list:
            # Append the type and priority to the result list
            result.append([item[1], priority])

    return result

def analyze_stats_data(data):
    # Initialize counters for each category
    counts = {
        "OPTIMIZATION": [0, 0, 0],  # counts for low, medium, and high priority
        "NETWORK": [0, 0, 0],
        "EXECUTION": [0, 0, 0]
    }
    
    # Iterate over each list (low, medium, high priority)
    for priority_level, priority_data in enumerate(data):
        for item in priority_data:
            if len(item) > 1:
                category = item[1]
                if category in counts:
                    counts[category][priority_level] += 1
    
    # Extract counts for each category
    optimization_alerts = counts["OPTIMIZATION"]
    network_alerts = counts["NETWORK"]
    execution_alerts = counts["EXECUTION"]
    
    return optimization_alerts, network_alerts, execution_alerts




def create_pie_chart(title,values):
    total = sum(values)
    if total!=0:
        percentage1 = (values[0] / total) * 100 
        percentage2 = (values[1] / total) * 100 
        percentage3 = (values[2] / total) * 100
        labels = ['Low', 'Medium', 'High']
        color_sequence = ['green', 'orange', 'red']
    else:
        values = [0]
        percentage1 = 100
        percentage2 = percentage3 = 0
        labels = ['No Value']
        color_sequence = ['grey']
    percentages = [percentage1, percentage2, percentage3]
    
    # Calculate the cumulative sum and angles
    cumsum = np.cumsum(percentages)
    total_percentage = sum(percentages)
    angles = (cumsum / total_percentage) * 180 

    
    # Create the arch chart
    fig = go.Figure()
    
    for i in range(len(values)):
        start_angle = angles[i-1] if i > 0 else 0
        end_angle = angles[i]
        
        fig.add_trace(go.Barpolar(
            r=[0.6],
            theta=[start_angle + (end_angle - start_angle)/2],
            width=[end_angle - start_angle],
            base=0.4,
            marker_color=color_sequence[i % len(color_sequence)],
            name=f"{labels[i]} ({percentages[i]:.2f}%)",
            text=[f"{labels[i]}<br>No. of Alerts: {values[i]}"],
            # textinfo="text",  # Use textinfo instead of textposition
            hoverinfo="text"
        ))
        
    # Customize the layout
    fig.update_layout(
        title=dict(
                    text=title,
                    x=0.5,
                    xanchor='center'
                ),
        polar=dict(
            radialaxis=dict(
                range=[0, 1], 
                showticklabels=False, 
                ticks='',
                showline=False,
                showgrid=False
            ),
            angularaxis=dict(
                showticklabels=False, 
                ticks='',
                showline=False,
                showgrid=False
            ),
            sector=[0, 180],
            bgcolor='rgba(0,0,0,0)'  # Transparent background
        ),
        showlegend=True,
        width=700,
        height=500,
        paper_bgcolor='rgba(0,0,0,0)',  # Transparent paper background
        plot_bgcolor='rgba(0,0,0,0)'    # Transparent plot background
    )
    return fig

try:
    drop_down_query_select = qprof.query_select_dropdown
except Exception as e:
    drop_down_query_select = widgets.HTML(f"<p style='color:red'>The following error occured: </p> <p> {e}</p>")

In [ ]:
# Info/Warning Box
import ipywidgets as widgets

def show_notification():
    close_button = widgets.Button(
        description="x",
        button_style="danger",
        layout={'width': '30px', 'height': '30px', 'margin': '0 0 0 10px'}
    )

    def close_notification(_):
        notification.close()
        close_button.close()

    close_button.on_click(close_notification)

    notification = widgets.HTML(
        value="<div style='background-color: #ffc107; padding: 10px; border-radius: 5px; display: flex; align-items: center;'>"
        "<strong>Important:</strong>&nbsp;If you do not see the trees, please click on the 'Tree Style' tab and then click on 'Apply'. This will refresh the tree."
        "</div>",
        layout={'margin': '10px 0'}
    )

    display(widgets.HBox([notification, close_button]))

show_notification()

In [ ]:
output_tree = widgets.Output()

with output_tree:
    logging.info(f'[Query Profile Tree Page] Trying to display the tree')
    if comparison:
        qprof_comparison.display()
    else:
        qprof.get_qplan_tree()
    logging.info(f'[Query Profile Tree Page] Finished trying to display the tree')
    
tabs = widgets.Tab()

child_1 = widgets.VBox([output_tree])
child_2 = widgets.VBox([drop_down_query_select, button_query_stats, output_query_stats, out_apply_button])
child_3 = engine_table_all
child_4 = widgets.VBox([button_get_queries, output_get_queries])
child_5 = widgets.VBox([plot_tabs_main])
child_6 = widgets.VBox([other_info])
if comparison:
    tabs.children = [child_1]
    tabs.titles = ["Tree"]
else:
    tabs.children = [child_1, 
                     child_2, 
                     child_3, 
                     child_4, 
                     child_5, 
                     child_6]
    tabs.titles = ["Tree", 
                   "Query Events", 
                   "Query Engine", 
                   "Transaction List",  
                   "Plots",
                   "Other info"]

# Define the observer function
def on_tab_change(change):
    if change['new'] == 2:  # Check if the first tab (index 0) is selected
        try:
            update_table('All', 'All')
        except Exception as e:
            pass
        # Stop observing after the first time
        tabs.unobserve(on_tab_change, names='selected_index')

# Attach the observer
tabs.observe(on_tab_change, names='selected_index')

display(tabs)

In [ ]:
query = widgets.Textarea(
    placeholder='Enter a SQL Query',
    description='SQL Query:',
    disabled=False,
    layout=widgets.Layout(width='700px')
)
button_run_query = widgets.Button(description="Run Query")
output_run_query = widgets.Output()
query_val = None

def on_button_clicked_run_query(b):
    output_run_query.clear_output(wait=True)
    global query_val, target_schema_val_2, key_val_2
    with output_run_query:
        query_val = query.value
        logging.info(f"""[Query Profile Tree Page] Running the following query manually :{query_val}.""")
        now = datetime.now(timezone.utc).astimezone()
        current_time = now.strftime("%Y-%m-%d %H:%M:%S %Z %z")
        display(HTML(f"<b>Last execution start (VerticaPyLab container [Docker] time with UTC offset):</b> {current_time}"))
        vp.vDataFrame(query_val).idisplay()

button_run_query.on_click(on_button_clicked_run_query)
manual_query = widgets.VBox([query, button_run_query, output_run_query])


______

In [ ]:
footer_widget = widgets.Checkbox(
    value=True,
    description='Footer on',
    disabled=False,
    indent=False
)
count_widget = widgets.Checkbox(
    value=False,
    description='Count on',
    disabled=False,
    indent=False
)
comma_widget = widgets.Checkbox(
    value=True,
    description='Insert commas for numbers',
    disabled=False,
    indent=False
)
percent_bar_widget = widgets.Checkbox(
    value=False,
    description='Percent bar',
    disabled=False,
    indent=False
)
max_cell_width_widget = widgets.BoundedIntText(
    value=280,
    min=250,
    max=350,
    step=1,
    description='Maximum cell width:',
    disabled=False,
    layout=widgets.Layout(width='300px')
)
max_cell_width_widget.style.description_width = '240px'
max_columns_widget = widgets.BoundedIntText(
    value=50,
    min=1,
    max=150,
    step=1,
    description='Maximum number of columns to display:',
    disabled=False,
    layout=widgets.Layout(width='300px')
)
max_columns_widget.style.description_width = '240px'
max_rows_widget = widgets.BoundedIntText(
    value=20000,
    min=1,
    max=4000,
    step=1,
    description='Maximum number of rows to display:',
    disabled=False,
    layout=widgets.Layout(width='300px')
)
max_rows_widget.style.description_width = '240px'
warning_text = """
<div style="background-color: #ffe5b3; color: #cc3300; padding: 10px; border: 1px solid #cc3300; border-radius: 5px;">
  <strong>Caution:</strong> These settings will reflect in your entire environment.
</div>
"""

warning_message = widgets.HTML(value=warning_text)
advanced_display_settings = widgets.VBox([
        warning_message,
        footer_widget,
        count_widget,
        comma_widget,
        percent_bar_widget,
        max_cell_width_widget,
        max_columns_widget,
        max_rows_widget,
    ],
)



update_advanced_display_options = widgets.Button(description="Update preferences")
output_advanced_display_options = widgets.Output()

def on_button_clicked_update_display_options(b):
    output_advanced_display_options.clear_output(wait=True)
    
    # Capture widget values
    with output_initial:
        global count_widget_val, footer_widget_val, comma_widget_val, percent_bar_widget_val, max_cell_width_widget_val, max_columns_widget_val, max_rows_widget_val
        count_widget_val = count_widget.value
        footer_widget_val = footer_widget.value
        comma_widget_val = comma_widget.value
        percent_bar_widget_val = percent_bar_widget.value
        max_cell_width_widget_val = max_cell_width_widget.value
        max_columns_widget_val = max_columns_widget.value
        max_rows_widget_val = max_rows_widget.value

    # Store the variables
    with output_advanced_display_options:
        %store count_widget_val footer_widget_val comma_widget_val percent_bar_widget_val max_cell_width_widget_val max_columns_widget_val max_rows_widget_val
        alert_text = """
        <div style="background-color: #e6f7ff; color: #007bff; padding: 10px; border: 1px solid #007bff; border-radius: 5px;">
          <strong>Alert:</strong> Changes have been updated. Please refresh the page to see them.
        </div>
        """
        display(widgets.HTML(value=alert_text))

update_advanced_display_options.on_click(on_button_clicked_update_display_options)
advanced_display_settings_widget = widgets.VBox([advanced_display_settings, update_advanced_display_options, output_advanced_display_options])

In [ ]:
explain_warning = widgets.HTML("""
<style>
.warning {
  padding: 5px;
  background-color: orange;
  color: white;
}
</style>
<div class="warning">
  <strong>Warning:</strong> Correct EXPLAIN output depends on being connected to the same database where the query was run, or an equivalent database.
</div>
""")

if comparison:
    button_run_explain_plan_1 = widgets.Button(description="Run Explain Plan")
    explain_output_1 = widgets.Output()
    try:
        explain_dropdown_options_1 = [i for i in range(len(qprof1.transactions))]
        explain_dropdown_disabled_1 = False
    except Exception as e:
        explain_dropdown_options_1 = []
        explain_dropdown_disabled_1 = True
        
    explain_dropdown_1 = widgets.Dropdown(
        options=explain_dropdown_options_1,
        description='Select Query Index',
        disabled=explain_dropdown_disabled_1,
        layout={'width': "300px"}
    )
    explain_dropdown_1.style.description_width = '130px'
    
    def on_button_clicked_run_explain_plan_1(b):
        explain_output_1.clear_output()
        with explain_output_1:
            qprof1.set_position(explain_dropdown_1.value)
            qprof1.get_qplan_explain()

    button_run_explain_plan_1.on_click(on_button_clicked_run_explain_plan_1)
    
    button_run_explain_plan_2 = widgets.Button(description="Run Explain Plan")
    explain_output_2 = widgets.Output()
    try:
        explain_dropdown_options_2 = [i for i in range(len(qprof2.transactions))]
        explain_dropdown_disabled_2 = False
    except Exception as e:
        explain_dropdown_options_2 = []
        explain_dropdown_disabled_2 = True
        
    explain_dropdown_2 = widgets.Dropdown(
        options=explain_dropdown_options_2,
        description='Select Query Index',
        disabled=explain_dropdown_disabled_2,  # Updated to use explain_dropdown_disabled_2
        layout={'width': "300px"}
    )
    explain_dropdown_2.style.description_width = '130px'
    
    def on_button_clicked_run_explain_plan_2(b):
        explain_output_2.clear_output()
        with explain_output_2:
            qprof2.set_position(explain_dropdown_2.value)
            qprof2.get_qplan_explain()
    
    button_run_explain_plan_2.on_click(on_button_clicked_run_explain_plan_2)
    left_side = widgets.VBox([explain_dropdown_1, button_run_explain_plan_1, explain_output_1])
    left_side.layout.width = '50%'
    right_side = widgets.VBox([explain_dropdown_2, button_run_explain_plan_2, explain_output_2])
    right_side.layout.width = '50%'

    vertica_border_line = widgets.HTML("""
    <style>
        .vertical-line {
            border-left: 2px solid lightgrey; /* Adjust thickness and color as needed */
            height: 500px; /* Adjust height as needed */
        }
    </style>
    </head>
    <body>
    
    <div class="vertical-line"></div>
    
    </body>
    """)    
    explain_plan_widget = widgets.HBox([left_side,vertica_border_line, right_side])
    explain_plan_widget = widgets.VBox([explain_warning, explain_plan_widget])
else:
    button_run_explain_plan = widgets.Button(description="Run Explain Plan")
    explain_output = widgets.Output()
    try:
        explain_dropdown_options = [i for i in range(len(qprof.transactions))]
        explain_dropdown_disabled = False
    except Exception as e:
        explain_dropdown_options = []
        explain_dropdown_disabled = True
        
    explain_dropdown = widgets.Dropdown(
        options=explain_dropdown_options,
        description='Select Query Index',
        disabled=explain_dropdown_disabled,
        layout={'width': "300px"}
    )
    explain_dropdown.style.description_width = '130px'
    
    def on_button_clicked_run_explain_plan(b):
        explain_output.clear_output()
        with explain_output:
            qprof.set_position(explain_dropdown.value)
            qprof.get_qplan_explain()
    
    button_run_explain_plan.on_click(on_button_clicked_run_explain_plan)
    explain_plan_widget = widgets.VBox([explain_warning, explain_dropdown, button_run_explain_plan, explain_output]) 
    

In [ ]:
consistency_check_out = widgets.Output()
consistency_check_button = widgets.Button(description="Run Checks")

def on_button_clicked_consistency_check_button(b):
    consistency_check_button.disabled = True
    consistency_check_out.clear_output(wait=True)
    with consistency_check_out:
        try:
            if comparison:
                qprof1._check_v_table(ignore_operators_check=False)
                qprof2._check_v_table(ignore_operators_check=False)
            else:
                qprof._check_v_table(ignore_operators_check=False)
        except Exception as e:
            print(e)
    consistency_check_button.disabled = False

consistency_check_button.on_click(on_button_clicked_consistency_check_button)
consistency_check_widget_box = widgets.VBox([consistency_check_button, consistency_check_out])

In [ ]:
accordion = widgets.Accordion(children=
                              [
                                manual_query, 
                                explain_plan_widget, 
                                advanced_display_settings_widget, 
                                consistency_check_widget_box,  
                                output_initial
                              ], 
                              titles=(
                                  'Manual Queries', 
                                  'Explain Plan', 
                                  'Advanced vDataFrame Output Settings',
                                  'Consistency Checks',
                                  'Execution/Error Details')
                             )
logging.info(f'[Query Profile Tree Page] Trying to display the error log at the bottom of the page')
accordion


In [ ]:
%%html
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Version Number</title>
    <style>
        .version {
            font-family: Arial, sans-serif;
            font-size: 14px;
            color: #555;
            padding: 5px 10px;
            background-color: #f9f9f9;
            border: 1px solid #ddd;
            border-radius: 4px;
            display: inline-block;
            margin-top: 10px;
        }
    </style>
</head>
<body>
    <div class="version">
        Page version update date: 10/18/2024
    </div>
</body>
</html>